In [1]:
import requests
import pandas as pd

In [2]:
#CareerOneStop/NLX
#First step is to gather job list; each job IDs will then be queried for specific job details/descriptions in the second step
#The maximum number of results is 500; script must iterate through pages to aggregate all the job IDs.
api_token = '2gaZEtABi5pxX1tFQeRgudXuWheouJKgYemjwUO58EH15WEKQJGw4NRXeDpbyH+r91fmkmv2ycO9uRH16yBIsQ=='    
user_id = 'cnzHhxx5yqwWiUl'
query_term = 'cybersecurity'
days = '0' #All days
start_page = 1
end_page = 500

url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
headers = {'Authorization': 'Bearer ' + api_token,
 'Content-Type': 'application/json'}
results = requests.get(url, headers=headers).json()


In [3]:
results

{'Jobcount': '50037',
 'Jobs': [{'JvId': '7C2321CECB83415B9D1D9854F8867997206',
   'JobTitle': 'Cybersecurity',
   'Company': 'US Navy',
   'AccquisitionDate': '2022-1-28 9:43 AM',
   'URL': 'https://de.jobsyn.org/7C2321CECB83415B9D1D9854F8867997206',
   'Location': 'Colorado Springs, Colorado',
   'Fc': 'False'},
  {'JvId': '89A5F5B490BA43EF816D7192BB369A1A206',
   'JobTitle': 'Cybersecurity',
   'Company': 'Idaho Department of Labor',
   'AccquisitionDate': '2022-1-29 7:05 AM',
   'URL': 'https://de.jobsyn.org/89A5F5B490BA43EF816D7192BB369A1A206',
   'Location': 'Nampa, Idaho',
   'Fc': 'False'},
  {'JvId': 'F4B11A16CE5740A898C929513BA7AFE5206',
   'JobTitle': 'Cybersecurity',
   'Company': 'US Navy',
   'AccquisitionDate': '2021-9-22 12:08 PM',
   'URL': 'https://de.jobsyn.org/F4B11A16CE5740A898C929513BA7AFE5206',
   'Location': 'Detroit, Michigan',
   'Fc': 'False'},
  {'JvId': '6640417C13B24BBDBEDA468648E26A8C206',
   'JobTitle': 'Cybersecurity',
   'Company': 'Arizona Department 

In [4]:
#Extract job IDs and append to master list
job_list = []
for i, value in enumerate(results['Jobs']):
    job_list.append(results['Jobs'][i]['JvId'])

In [11]:
len(job_list), job_list

(500,
 ['7C2321CECB83415B9D1D9854F8867997206',
  '89A5F5B490BA43EF816D7192BB369A1A206',
  'F4B11A16CE5740A898C929513BA7AFE5206',
  '6640417C13B24BBDBEDA468648E26A8C206',
  'EB437BF5A6164CFDAA3A7E1755707369206',
  '834B5A0808FB4F08BD812F4A3E1F23DD206',
  '63F6C597C0CD43F394925E42724B59B1206',
  'ECC56D451BD145CABD98CCB8248D614F206',
  'F6D7509040444F1C92D3406481C332D2206',
  '78723A260F9341F6AD8FBC10255E99C3206',
  '6F0A908AC16D423CB515B1A18952C5E6206',
  '7C8BBC319F9243F8BA6C30481EAFF2BC206',
  '14164C82B5A0416BA5FBC63BE458494B206',
  'B906F2FE2A0A43599161510D04D2AB72206',
  'CC9AD5002539459D932B5BA29AF798FC206',
  '63F9CC2AED814368AF5E9B93570F5B7D206',
  '4EA94C1C68A0439CA93BF1F362547723206',
  'FC645122636E4855B546105A2FB1DA2E206',
  '359AEC970DB744A69962ADC4D080F00F206',
  'C927DE150A84428AB2EFB5D28714E1A5206',
  '4E1B9D4C9BB243A1B4BA732925B89902206',
  'E64647EE0DB841A0925B69E85F6C8D92206',
  '6E94E29AF2C3491F850A64F0271F096C206',
  '2865842C39DB4A8E9BF87F22F172258D206',
  '3949D20

In [6]:
#Retrieve job details using job IDs; this example takes the first job ID in job_list
url_getjob = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{job_list[1]}?isHtml=true&enableMetaData=true'
results = requests.get(url_getjob, headers=headers).json()

In [7]:
results

{'JobTitle': 'Cybersecurity',
 'URL': 'https://de.jobsyn.org/89A5F5B490BA43EF816D7192BB369A1A',
 'Company': 'Idaho Department of Labor',
 'Location': 'Nampa, Idaho',
 'DatePosted': '2022-1-29 7:05 AM',
 'Description': '<p>This job was posted by https://idahoworks.gov : For more information, please see: https://idahoworks.gov/jobs/1655485 &&&&ABOUT&&&&&&As a Cryptologic Technician, you are one of the world’s greatest problem-solvers. We’re looking for people with sharp minds, whose brains are built for picking out patterns most will miss.&&a href="https://www.navy.com/start?activity=1257145" target="_blank" rel="noopener"&&&APPLY HERE.&&&/a&&&&&&&CRYPTOLOGY JOBS IN THE NAVY&&&&&&&&—CRYPTOLOGIC TECHNICIAN COLLECTION&&&&&&Much about your role is classified. What we can tell you is this: you will intercept signals to provide tactical and strategic intelligence, technical guidance and information to ships and special warfare units.&&&&&&—CRYPTOLOGIC TECHNICIAN INTERPRETIVE&&&&&&Languages ar

In [10]:
#Most likely fields to retain
results['JobTitle'], results['Company'], results['Description']

('Cybersecurity',
 'Idaho Department of Labor',
 '<p>This job was posted by https://idahoworks.gov : For more information, please see: https://idahoworks.gov/jobs/1655485 &&&&ABOUT&&&&&&As a Cryptologic Technician, you are one of the world’s greatest problem-solvers. We’re looking for people with sharp minds, whose brains are built for picking out patterns most will miss.&&a href="https://www.navy.com/start?activity=1257145" target="_blank" rel="noopener"&&&APPLY HERE.&&&/a&&&&&&&CRYPTOLOGY JOBS IN THE NAVY&&&&&&&&—CRYPTOLOGIC TECHNICIAN COLLECTION&&&&&&Much about your role is classified. What we can tell you is this: you will intercept signals to provide tactical and strategic intelligence, technical guidance and information to ships and special warfare units.&&&&&&—CRYPTOLOGIC TECHNICIAN INTERPRETIVE&&&&&&Languages are more than just communication—they’re cultural codes that need to be analyzed and, in some cases, broken. The Defense Language School in Monterey, California will fully